# torch基础

In [4]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(
    nn.Linear(20, 256),
    nn.ReLU(),
    nn.Linear(256,10)
)

X = torch.rand(2, 20)
net(X)

tensor([[-0.1357, -0.0125,  0.0046,  0.0142,  0.1070, -0.0520,  0.0111, -0.0525,
          0.0102, -0.0122],
        [-0.0865, -0.0259,  0.0199,  0.0435,  0.2222, -0.1078,  0.1453, -0.0445,
         -0.0377,  0.1575]], grad_fn=<AddmmBackward0>)

In [6]:
class Mlp(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
    
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))
    
net = Mlp()
net(X)

tensor([[-0.0062,  0.1070, -0.0068,  0.1097,  0.1255,  0.1197, -0.0317, -0.0284,
         -0.0027,  0.1359],
        [-0.0574,  0.2145,  0.0707, -0.0774,  0.1113,  0.3319, -0.1210, -0.1049,
         -0.1229,  0.2157]], grad_fn=<AddmmBackward0>)

In [10]:
class MySequential(nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        # enumerate是Python的内置函数，用于遍历可迭代对象（如列表、元组）时，同时获取元素的索引和值
        for idx, module in enumerate(args):  # idx是索引，module是对应的模块
            self._modules[str(idx)] = module  # 修正：str(idx) 而不是 str[idx]
            
    def forward(self, X):   # 修正：forward 方法需要接收输入 X
        for block in self._modules.values():
            X = block(X)
        return X
    
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.2297,  0.0415,  0.0562, -0.2076,  0.0330, -0.0943,  0.0129, -0.0502,
         -0.1394,  0.0710],
        [-0.4184,  0.0044, -0.1194, -0.2837,  0.1500, -0.2105,  0.0254, -0.0534,
         -0.3026, -0.0459]], grad_fn=<AddmmBackward0>)

In [11]:
class FixedHiddenMlp(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.rand_weight = torch.rand((20,20), requires_grad=False)
        self.linear = nn.Linear(20,20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) +1)
        X = self.linear(X)
        
        while X.abs().sum() >1:
            X/=2
        
        return X.sum()
    
net = FixedHiddenMlp()
net(X)

tensor(0.1705, grad_fn=<SumBackward0>)

In [12]:
class NestMlp(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(20, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        
        self.linear = nn.Linear(32,16)
        
    def forward(self,X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMlp(), nn.Linear(16, 20,), FixedHiddenMlp())

chimera(X)

tensor(-0.0360, grad_fn=<SumBackward0>)